In [1]:
import numpy as np
import os
import pandas as pd
#from arcpy import env
#from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

sSuffix = '_2024_04'

In [2]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
dirWork      = os.getcwd()
dirData      = os.path.join(dirWork, r'data')
dirTDM       = os.path.join(dirData, r'tdm\1_WF')
dirResults   = os.path.join(dirWork, r'results')
dirLayers    = os.path.join(dirResults, r'layers')
dirProcessed = os.path.join(dirWork, r'data\tdm\1_WF')

In [4]:
dDirections = {
     'Dir'     : ['D1'   ,'D2'   ],
     'DirDesc' : ['NB/EB','SB/WB']
     
}
df_Directions = pd.DataFrame(dDirections)


In [5]:
shp_Forecasts = os.path.join(dirData,  r'forecasts\Traffic-Volume-Historic-and-Forecast.shp')
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)

In [6]:
df_Scenarios       = pd.read_csv(os.path.join(dirResults, r'scenarios.csv'))
df_VCGroup_Seasons = pd.read_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'))
df_VCGroup_DOWPeak = pd.read_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'))
df_VCGroups        = pd.read_csv(os.path.join(dirResults, r'vcgroups.csv'))

df_VCGroups_15  = df_VCGroups[['VCGroupCode']]
df_VCGroups_Hr  = df_VCGroups[['VCGroupCode']]
df_VCGroups_Prd = df_VCGroups[df_VCGroups['HRPCTOF'] == 'Prd'].copy()
df_VCGroups_Prd = df_VCGroups_Prd[['VCGroupCode']]
df_VCGroups_15 ['VCGroupCode'] = df_VCGroups_15['VCGroupCode'] + "15"
df_VCGroups_Hr ['VCGroupCode'] = df_VCGroups_Hr['VCGroupCode'] + "Hr"
df_VCGroups_Prd['VCGroupCode'] = df_VCGroups_Prd['VCGroupCode'] + "Prd"
df_VCGroups_Expanded = pd.concat([df_VCGroups_15,df_VCGroups_Hr,df_VCGroups_Prd])
    
display(df_Scenarios)
display(df_VCGroup_Seasons)
display(df_VCGroup_DOWPeak)
display(df_VCGroups_Expanded)


<ipython-input-6-a16000379118>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_VCGroups_15 ['VCGroupCode'] = df_VCGroups_15['VCGroupCode'] + "15"
<ipython-input-6-a16000379118>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_VCGroups_Hr ['VCGroupCode'] = df_VCGroups_Hr['VCGroupCode'] + "Hr"


,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
5,RTP50,2050 Fiscally-Constrained Scenario,2050,RTP,v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,RTP42,NaN
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
8,NB5028,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50
9,NB4232,2042 SE on a 2032 Network,2042,NoBuild32,v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed...,Yes,RTP32,RTP42


,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


,DOWPkCode,DOWPkName,DOWGROUP,PERIOD,HRPCTOF
0,WkAM,AM - Weekday,1-Weekday (Tu-Th),AM,Prd
1,WkMD,Midday - Weekday,1-Weekday (Tu-Th),MD,Prd
2,WkPM,PM - Weekday,1-Weekday (Tu-Th),PM,Prd


,VCGroupCode
0,AnnWkAM15
1,AnnWkMD15
2,AnnWkPM15
0,AnnWkAMHr
1,AnnWkMDHr
2,AnnWkPMHr
0,AnnWkAMPrd
1,AnnWkMDPrd
2,AnnWkPMPrd


In [7]:
df_VCGroups_Prd

,VCGroupCode
0,AnnWkAMPrd
1,AnnWkMDPrd
2,AnnWkPMPrd


In [8]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']
def ScenarioNameFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioName']


def DOWPkNameFromCode (sCode):
    return df_VCGroup_DOWPeak[df_VCGroup_DOWPeak['DOWPkCode'] == sCode].iloc[0]['DOWPkName']
def SeasonNameFromCode (sCode):
    return df_VCGroup_Seasons[df_VCGroup_Seasons['SeasonCode'] == sCode].iloc[0]['SeasonName']



#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

In [9]:
sCode = 'AvgWkPM15'
sCode = 'WkPM'
#DOWPkNameFromCode ()
#sCode
df_VCGroup_DOWPeak

,DOWPkCode,DOWPkName,DOWGROUP,PERIOD,HRPCTOF
0,WkAM,AM - Weekday,1-Weekday (Tu-Th),AM,Prd
1,WkMD,Midday - Weekday,1-Weekday (Tu-Th),MD,Prd
2,WkPM,PM - Weekday,1-Weekday (Tu-Th),PM,Prd


In [10]:
dfs_VC_MaxDirFG_wVCGroups =[]

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    dfs_VC_MaxDirFG_wVCGroups.append(pd.read_csv(os.path.join(dirResults, r'temp\3_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv')))


'Scenario Codes:'

Base19
Base23
TIP28
RTP32
RTP42
RTP50
NB3228
NB4228
NB5028
NB4232
NB5032
NB5042
UF32
UF42
UF50
UF50_MAG


In [11]:
dfs_VC_MaxDirFG_wVCGroups[0]

,SEGID,FT,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,2.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,2.0,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,2.0,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,2.0,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,3.0,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,4.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [12]:
sScenario = 'RTP32'
sVCGroup  = 'AnnWkPMPrd'

map_seg = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression = ("var v = $feature." + sVCGroup + ";"
                     "var ft = $feature.FT;"
                     "if      (v<0.7  && ft<10) { return 'class_a1'; }"
                     "else if (v<0.85 && ft<10) { return 'class_a2'; }"
                     "else if (v<1.0  && ft<10) { return 'class_a3'; }"
                     "else if (v<1.5  && ft<10) { return 'class_a4'; }"
                     "else if (v>=1.5 && ft<10) { return 'class_a5'; }"
                     "else if (v<0.7  && ft>10) { return 'class_f1'; }"
                     "else if (v<0.85 && ft>10) { return 'class_f2'; }"
                     "else if (v<1.0  && ft>10) { return 'class_f3'; }"
                     "else if (v<1.5  && ft>10) { return 'class_f4'; }"
                     "else if (v>=1.5 && ft>10) { return 'class_f5'; }"
                    )

#symbology for enrollment classes
uv = [{"value":"class_a1", "label":"Arterial Less than 0.70", "symbol":{"type":"esriSLS","color":[  0,255,  0,160], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a2", "label":"Arterial 0.70 to 0.85"  , "symbol":{"type":"esriSLS","color":[169,243,106], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a3", "label":"Arterial 0.85 to 1.00"  , "symbol":{"type":"esriSLS","color":[255,228,105], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a4", "label":"Arterial 1.00 to 1.50"  , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a5", "label":"Arterial More than 1.50", "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_f1", "label":"Freeway Less than 0.70" , "symbol":{"type":"esriSLS","color":[  0,255,  0,160], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f2", "label":"Freeway 0.70 to 0.85"   , "symbol":{"type":"esriSLS","color":[169,243,106], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f3", "label":"Freeway 0.85 to 1.00"   , "symbol":{"type":"esriSLS","color":[255,228,105], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f4", "label":"Freeway 1.00 to 1.50"   , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f5", "label":"Freeway More than 1.50" , "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":4.0,"style":"esriSLSSolid"}}]

idDisplay = idFromCode(sScenario)
sTitle = "Congestion: " + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7])

sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_VC_MaxDirFG_wVCGroups[idDisplay],on='SEGID')

sdf_Forecasts_wVC = sdf_Forecasts_wVC[['SEGID',sVCGroup,'FT','SHAPE']]


#define sdf layer
sdf_Forecasts_wVC.spatial.plot(map_widget = map_seg,renderer_type='u-a',unique_values=uv,arcade_expression=arcade_expression,default_symbol="",arcade_title="V/C Ratio Range")

#define map characteristics
map_seg.layout.height='800px'
map_seg.legend=True

#map title
display(Markdown('<h2><center>' + sTitle + '</center></h2>'))

#display map
map_seg

<h2><center>Congestion: 2032 Fiscally-Constrained Scenario - Annual PM - Weekday</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [13]:
sdf_Forecasts_wVC

,SEGID,AnnWkPMPrd,FT,SHAPE
0,0006_141.0,0.07,2.0,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,0006_146.9,0.07,2.0,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,0006_149.9,0.17,2.0,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,0006_150.6,0.16,2.0,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,0006_152.6,0.16,3.0,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...
4573,WFRC_8466,0.40,3.0,"{""paths"": [[[429935.1068000002, 4512218.204299..."
4574,WFRC_8467,0.71,2.0,"{""paths"": [[[415925.6900000004, 4495878.66], [..."
4575,WFRC_8471,0.52,4.0,"{""paths"": [[[415734.59509999957, 4481608.6051]..."
4576,WFRC_8473,0.69,5.0,"{""paths"": [[[417141.7167999996, 4545904.8543],..."


In [14]:
df_VC = pd.read_csv(os.path.join(dirResults, r'1_VC_' + sScenario + '.csv'))

In [15]:
sScenario

'RTP32'

In [16]:
df_VC_filtered = df_VC[(df_VC['SEGID']=='WFRC_8198') & (df_VC['VCGroupCode']==sVCGroup[0:7])]
pd.set_option('display.max_columns', None)
df_VC_filtered

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
28124,WFRC_8198,2,104814.2,Freeway,1.0,D1,2,33,Freeway,1730,0.4618,PM,0.2183,10.8,5.8,2032,89500,89500,9022.57913,Statewide,0,1000000,XX3,1.0,S00-Ann,9022,1.16,1-Weekday (Tu-Th),10436,2015-2019,PM,0.3691,Prd,Season,AnnWkPM,3,3851.9276,1.0506,3654,4046,1.06,1.17,0.95,4259,1.23
28125,WFRC_8198,2,104814.2,Freeway,1.0,D2,2,33,Freeway,1730,0.5382,PM,0.2183,9.0,4.5,2032,89500,89500,10515.27087,Statewide,0,1000000,XX3,1.0,S00-Ann,10515,1.16,1-Weekday (Tu-Th),12163,2015-2019,PM,0.3691,Prd,Season,AnnWkPM,3,4489.3633,1.0405,4218,4671,1.22,1.35,0.95,4917,1.42


In [17]:
try:
    df_ForecastAdj = pd.read_csv(os.path.join(dirResults, r'ForecastAdj_' + sScenario + '.csv'))
except Exception:
    display ('No forecast adjustment')
    pass

'No forecast adjustment'

In [18]:
try:
    df_ForecastAdj_filtered = df_ForecastAdj[(df_ForecastAdj['SEGID']=='0085_011.3')]
    df_ForecastAdj_filtered
except Exception:
    display ('No forecast adjustment')
    pass  

'No forecast adjustment'

# Create Layers

In [19]:
dirLayers

'e:\\GitHub\\V-over-C-Calculations\\results\\layers'

In [20]:
for (idx, row) in df_Scenarios.iterrows():

    sScenario = row.loc['ScenarioCode']

    idLoc = idFromCode(sScenario)

    sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_VC_MaxDirFG_wVCGroups[idLoc],on='SEGID')

    sdf_Forecasts_wVC['Scenario'] = sScenario
    
    outFields = ['Scenario','SEGID','FT','SHAPE'] + list(df_VCGroups_Expanded.VCGroupCode.unique())
    
    sdf_Forecasts_wVC = sdf_Forecasts_wVC[outFields]
    
    sdf_Forecasts_wVC = sdf_Forecasts_wVC.fillna(0)
    #display(sdf_Forecasts_wVC)

#    try:
    sdf_Forecasts_wVC.spatial.to_featureclass(location=os.path.join(dirLayers,'VC_' + sScenario + sSuffix + '.shp'), sanitize_columns=False)
    display("Congestion: " + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7]))
#    except Exception:
#        display ('FAIL:" + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7])))
#        pass
 

<ipython-input-20-628c407088e8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sdf_Forecasts_wVC['Scenario'] = sScenario


'Congestion: 2019 Base - Annual PM - Weekday'

<ipython-input-20-628c407088e8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sdf_Forecasts_wVC['Scenario'] = sScenario


'Congestion: 2023 Base - Annual PM - Weekday'

<ipython-input-20-628c407088e8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sdf_Forecasts_wVC['Scenario'] = sScenario


...

'Congestion: 2042 Needs-Based Scenario - Annual PM - Weekday'

<ipython-input-20-628c407088e8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sdf_Forecasts_wVC['Scenario'] = sScenario


'Congestion: 2050 Needs-Based Scenario - Annual PM - Weekday'

<ipython-input-20-628c407088e8>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sdf_Forecasts_wVC['Scenario'] = sScenario


'Congestion: 2050 Needs-Based Scenario - MAG - Annual PM - Weekday'

# Segment Data for WebApp

In [21]:
dfs_VC = []

for (idx, row) in df_Scenarios.iterrows():

    display(row.loc['ScenarioCode'])
    
    df_VC = pd.read_csv(os.path.join(dirResults, r'1_VC_' + row.loc['ScenarioCode'] + '.csv'))
    
    #some older files may still use old column names, rename them here to be consistent moving forward
    df_VC = df_VC.rename(columns={"MDPERCENT": "MDPCT", "HVPERCENT": "HVPCT"})
    
    dfs_VC.append(df_VC)
    

'Base19'

'Base23'

'TIP28'

'RTP32'

'RTP42'

...

'NB5042'

'UF32'

'UF42'

'UF50'

'UF50_MAG'

In [22]:
dReportingColumns_Seg           = ['SEGID'     ,'ForecastAADT'     ,'ForecastAADTAdj'       ]
dReportingColumns_Seg_Names     = ['Segment ID','RTP Forecast AADT','Scenario Forecast AADT']
dReportingColumns_Seg_jsonNames = ['S'         ,'F'                ,'Fa']

dReportingColumns_Data           = ['VCGroupCode'  ,'FUNCGROUP'       ,'FGFAC'                  ,'SiteGroupSeason'  ,'SeasonFactor' ,'DOWFactor'         ,'PRDFAC'       ,'VolPkHrPct',     'Dir'      ,'DFAC'            ,'MDPCT'         ,'HVPCT'        ,'TrkFac'      ,'PHF','LANES','AREATYPE' ,'FT'             ,'Capacity'      ,'PrdPCEFlow'       ,'PkHrPCEFlow'         ,'Pk15PCEFlow'              ,'PrdVC'     ,'PkHrVC'       ,'Pk15VC'         ]
dReportingColumns_Data_Names     = ['VC Group Code','Functional Group','Functional Group Factor','Season Group Code','Season Factor','Day-of-Week Factor','Period Factor','Max Hour Factor','Direction','Direction Factor','Medium Truck %','Heavy Truck %','Truck Factor','PHF','Lanes','Area Type','Functional Type','Capacity (vph)','Period Flow (vph)','Peak Hour Flow (vph)','Peak 15-Minute Flow (vph)','Period V/C','Peak Hour V/C','Peak 15-Min V/C']
dReportingColumns_Data_jsonNames = ['VCG'          ,'FG'              ,'FGF'                    ,'SG'               ,'SF'           ,'DOW'               ,'PF'           ,'HF'             ,'D'        ,'DF'              ,'MD'            ,'HV'           ,'TF'          ,'PHF','LN'   ,'AT'       ,'FT'             ,'CP'            ,'PrdF'             ,'PkF'                 ,'15F'                      ,'PrdVC'     ,'PkVC'         ,'15VC'           ]

dReportingColumns_All = dReportingColumns_Seg + dReportingColumns_Data
dReportingColumns_All_jsonNames = dReportingColumns_Seg_jsonNames + dReportingColumns_Data_jsonNames
dReportingColumns_All_jsonNames


['S',
 'F',
 'Fa',
 'VCG',
 'FG',
 'FGF',
 'SG',
 'SF',
 'DOW',
 'PF',
 'HF',
 'D',
 'DF',
 'MD',
 'HV',
 'TF',
 'PHF',
 'LN',
 'AT',
 'FT',
 'CP',
 'PrdF',
 'PkF',
 '15F',
 'PrdVC',
 'PkVC',
 '15VC']

In [23]:
#exportscenariocode='RTP50'

for index, df_VC in enumerate(dfs_VC):

    sScenarioCode = df_Scenarios['ScenarioCode'][index]
    
#    if (sScenarioCode==exportscenariocode):

    display (sScenarioCode)

    #dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size().reset_index(name='Freq')
    #dfOutputSegs = dfOutputSegs[dReportingColumns_Seg]
    
    df_VC['Capacity'   ] =        df_VC['CAP1HL'     ] * df_VC['LANES']
    
    df_VC['Capacity'   ] = round((df_VC['Capacity'   ]/10),0)*10
    df_VC['PrdPCEFlow' ] = round((df_VC['PrdPCEFlow' ]/10),0)*10
    df_VC['PkHrPCEFlow'] = round((df_VC['PkHrPCEFlow']/10),0)*10
    df_VC['Pk15PCEFlow'] = round((df_VC['Pk15PCEFlow']/10),0)*10
    
    #if df_VC['ForecastAADT'   ].dtype != 'str' : df_VC['ForecastAADT'   ].astype('float64')
    #if df_VC['ForecastAADTAdj'].dtype != 'str' : df_VC['ForecastAADTAdj'].astype('float64')
    #if df_VC['Capacity'       ].dtype != 'str' : df_VC['Capacity'       ].astype('float64')
    #if df_VC['FGFAC'          ].dtype != 'str' : df_VC['FGFAC'          ].astype('float64')
    #if df_VC['PRDFAC'         ].dtype != 'str' : df_VC['PRDFAC'         ].astype('float64')
    #if df_VC['DOWFactor'      ].dtype != 'str' : df_VC['DOWFactor'      ].astype('float64')
    #if df_VC['DFAC'           ].dtype != 'str' : df_VC['DFAC'           ].astype('float64')
    #if df_VC['Capacity'       ].dtype != 'str' : df_VC['Capacity'       ].astype('float64')
    #if df_VC['PrdPCEFlow'     ].dtype != 'str' : df_VC['PrdPCEFlow'     ].astype('float64')
    #if df_VC['PkHrPCEFlow'    ].dtype != 'str' : df_VC['PkHrPCEFlow'    ].astype('float64')
    #if df_VC['Pk15PCEFlow'    ].dtype != 'str' : df_VC['Pk15PCEFlow'    ].astype('float64')
    #if df_VC['Capacity'       ].dtype != 'str' : df_VC['Capacity'       ].astype('float64')
    #if df_VC['PrdPCEFlow'     ].dtype != 'str' : df_VC['PrdPCEFlow'     ].astype('float64')
    #if df_VC['PkHrPCEFlow'    ].dtype != 'str' : df_VC['PkHrPCEFlow'    ].astype('float64')
    #if df_VC['Pk15PCEFlow'    ].dtype != 'str' : df_VC['Pk15PCEFlow'    ].astype('float64')
    #if df_VC['MDPCT'          ].dtype != 'str' : df_VC['MDPCT'          ].astype('float64')
    #if df_VC['HVPCT'          ].dtype != 'str' : df_VC['HVPCT'          ].astype('float64')
    #if df_VC['TrkFac'         ].dtype != 'str' : df_VC['TrkFac'         ].astype('float64')
    #if df_VC['PrdVC'          ].dtype != 'str' : df_VC['PrdVC'          ].astype('float64')
    #if df_VC['PkHrVC'         ].dtype != 'str' : df_VC['PkHrVC'         ].astype('float64')
    #if df_VC['Pk15VC'         ].dtype != 'str' : df_VC['Pk15VC'         ].astype('float64')

    df_VC['ForecastAADT'   ] = df_VC.apply(lambda x: "{:,.0f}".format(x['ForecastAADT'   ]), axis=1)
    df_VC['ForecastAADTAdj'] = df_VC.apply(lambda x: "{:,.0f}".format(x['ForecastAADTAdj']), axis=1)

    #dfOutputSegs.columns  = dReportingColumns_Seg_jsonNames
    #dfOutputSegs.to_json(os.path.join(dirResults, 'WebAppData\\segment_vcdata\\_scenariosegs_' + sScenarioCode + ".json"),orient='records')

    df_VC['FGFAC'          ] = df_VC.apply(lambda x: "{:.3}"  .format(x['FGFAC'      ]), axis=1)    
    df_VC['PRDFAC'         ] = df_VC.apply(lambda x: "{:.3}"  .format(x['PRDFAC'     ]), axis=1)    
    df_VC['VolPkHrPct'     ] = df_VC.apply(lambda x: "{:.3}"  .format(x['VOLPKHRPCT' ]), axis=1)    
    df_VC['SeasonFactor'   ] = df_VC.apply(lambda x: "{:.3}"  .format(x['SsnFactor'  ]), axis=1)
    df_VC['DOWFactor'      ] = df_VC.apply(lambda x: "{:.3}"  .format(x['DOWFactor'  ]), axis=1)
    df_VC['DFAC'           ] = df_VC.apply(lambda x: "{:.3}"  .format(x['DFAC'       ]), axis=1)

    df_VC['Capacity'       ] = df_VC.apply(lambda x: "{:,.0f}".format(x['Capacity'   ]), axis=1)
    df_VC['PrdPCEFlow'     ] = df_VC.apply(lambda x: "{:,.0f}".format(x['PrdPCEFlow' ]), axis=1)
    df_VC['PkHrPCEFlow'    ] = df_VC.apply(lambda x: "{:,.0f}".format(x['PkHrPCEFlow']), axis=1)
    df_VC['Pk15PCEFlow'    ] = df_VC.apply(lambda x: "{:,.0f}".format(x['Pk15PCEFlow']), axis=1)
    
    df_VC['MDPCT'          ] = df_VC.apply(lambda x: "{:.1f}" .format(x['MDPCT'      ]), axis=1)
    df_VC['HVPCT'          ] = df_VC.apply(lambda x: "{:.1f}" .format(x['HVPCT'      ]), axis=1)

    df_VC['TrkFac'         ] = df_VC.apply(lambda x: "{:.3f}" .format(x['TrkFac'     ]), axis=1)
    df_VC['PrdVC'          ] = df_VC.apply(lambda x: "{:.2f}" .format(x['PrdVC'      ]), axis=1)
    df_VC['PkHrVC'         ] = df_VC.apply(lambda x: "{:.2f}" .format(x['PkHrVC'     ]), axis=1)
    df_VC['Pk15VC'         ] = df_VC.apply(lambda x: "{:.2f}" .format(x['Pk15VC'     ]), axis=1)  

    df_VC['SiteGroupSeason'] = df_VC['STATIONGROUP']
    #    #fVCforSet = df_VC.groupby(['SEGID','VCGroupCode'], as_index=False).size().reset_index(name='Freq')
    #    
    #    for index, row in dfOutputSegs.iterrows():
    #
    #        sSegID = row['SegID']
    #        #sVCGroupCode = row['VCGroupCode']
    #          
    #        #dfOutputData = df_VC[(df_VC['SEGID'] == sSegID) & (df_VC['VCGroupCode'] == sVCGroupCode)]
    #        
    #        dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]
    #
    #        #sFilename = sScenarioCode + '_' + sSegID + '_ '+ sVCGroupCode + ".json"
    #        sFilename = sScenarioCode + '_' + sSegID + ".json"
    #        #display(sFilename)
    #        
    #        dfOutputData = dfOutputData[dReportingColumns_Data]
    #        
    #        dfOutputData.columns  = dReportingColumns_Data_jsonNames
    #        
    #        dfOutputData.to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records')
    

'Base19'

'Base23'

'TIP28'

'RTP32'

'RTP42'

...

'NB5042'

'UF32'

'UF42'

'UF50'

'UF50_MAG'

In [24]:
df_VC.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC', 'PHF', 'Pk15PCEFlow',
       'Pk15VC', 'Capacity', 'VolPkHrPct', 'SeasonFactor', 'SiteGroupSeason'],
      dtype='object')

## Segment Files

In [25]:
# INDIVIDUAL FILES PER SCENARIO

arrnew = []

for index, df_VC in enumerate(dfs_VC):

    sScenarioCode = df_Scenarios['ScenarioCode'][index]
    
#    if (sScenarioCode==exportscenariocode):

    display (sScenarioCode)

    dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size()#.reset_index(name='Freq')

    folder_path = os.path.join(dirResults, "WebAppData\\segment_vcdata\\")
    sFilename = sScenarioCode + ".json"
    
    os.makedirs(folder_path, exist_ok=True) 
    # which will not raise an error if the `path` already exists and it
    # will recursively create the paths, if the preceding path doesn't exist
    
    text_file = open(os.path.join(folder_path, sFilename), "w")
    
    arrnew = []
    strnew_combined = ""
    
    for index, row in dfOutputSegs.iterrows():

        sSegID = row['SEGID']

        dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]

        dfOutputDataNew = dfOutputData[dReportingColumns_All].copy()
        dfOutputDataNew.columns = dReportingColumns_All_jsonNames       

#            group = merge_hr_anr.groupby('EMPCODE').groups
#            d = {'EMPCODE': list(group.keys())[0], 'Indicators': [{'IndicatorName': merge_hr_anr.loc[i, 'Indicator']} for i in list(group.values())[0].unique()]}
#            d['Indicators'] = list(map(dict,sorted(set(map(lambda x: tuple(x.items()),d['Indicators'])), key=list(map(lambda x: tuple(x.items()),d['Indicators'])).index)))
#            d['Performance'] = [{i['IndicatorName']: merge_hr_anr.loc[merge_hr_anr['Indicator'].eq(i['IndicatorName']), 'CustNAME'].dropna().tolist()} for i in d['Indicators']]
#            print(d)
#            
#            j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
#                    .apply(lambda x: x[dReportingColumns_Data_jsonNames]
#                    .to_dict('r'))
#                    .reset_index()
#                    .rename(columns={0:'VCData'})
#                    .to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records'))#, indent=2))

        # manual json creation code as created below
        df = dfOutputDataNew[dReportingColumns_Seg_jsonNames].drop_duplicates()

        out = df.to_json(orient='records')

        j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
                            .apply(lambda x: x[dReportingColumns_Data_jsonNames])
            ).to_dict('records')                   
        #ranslation= {39: None}

        strnew = out[1:-2]+',"VCData":' + str(j)+"}"

        arrnew.append(strnew)

        #
        #display(j)

        #break
        
    strnew_combined = '[' + ",".join(arrnew) + ']'
    n = text_file.write(strnew_combined.replace("'",'"').replace(" ",""))
    text_file.close()

'Base19'

'Base23'

'TIP28'

'RTP32'

'RTP42'

...

'NB5042'

'UF32'

'UF42'

'UF50'

'UF50_MAG'

In [26]:
## INDIVIDUAL FILES PER SEGMENT
#
###### NOT CURRENTLY USING, BUT WANT TO PRESERVE CODE #####
#
#
#
#for index, df_VC in enumerate(dfs_VC):
#
#    sScenarioCode = df_Scenarios['ScenarioCode'][index]
#    
##    if (sScenarioCode==exportscenariocode):
#
#    display (sScenarioCode)
#
#    dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size()#.reset_index(name='Freq')
#
#    folder_path = os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sScenarioCode)
#    
#    os.makedirs(folder_path, exist_ok=True) 
#    # which will not raise an error if the `path` already exists and it
#    # will recursively create the paths, if the preceding path doesn't exist
#    
#    for index, row in dfOutputSegs.iterrows():
#
#        sSegID = row['SEGID']
#
#        dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]
#
#        dfOutputDataNew = dfOutputData[dReportingColumns_All].copy()
#        dfOutputDataNew.columns = dReportingColumns_All_jsonNames
#
#        sFilename = sScenarioCode + '_' + sSegID + ".json"
#
##            group = merge_hr_anr.groupby('EMPCODE').groups
##            d = {'EMPCODE': list(group.keys())[0], 'Indicators': [{'IndicatorName': merge_hr_anr.loc[i, 'Indicator']} for i in list(group.values())[0].unique()]}
##            d['Indicators'] = list(map(dict,sorted(set(map(lambda x: tuple(x.items()),d['Indicators'])), key=list(map(lambda x: tuple(x.items()),d['Indicators'])).index)))
##            d['Performance'] = [{i['IndicatorName']: merge_hr_anr.loc[merge_hr_anr['Indicator'].eq(i['IndicatorName']), 'CustNAME'].dropna().tolist()} for i in d['Indicators']]
##            print(d)
##            
##            j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
##                    .apply(lambda x: x[dReportingColumns_Data_jsonNames]
##                    .to_dict('r'))
##                    .reset_index()
##                    .rename(columns={0:'VCData'})
##                    .to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records'))#, indent=2))
#
#        # manual json creation code as created below
#        df = dfOutputDataNew[dReportingColumns_Seg_jsonNames].drop_duplicates()
#
#        out = df.to_json(orient='records')
#
#        j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
#                            .apply(lambda x: x[dReportingColumns_Data_jsonNames])
#            ).to_dict('records')                   
#        #ranslation= {39: None}
#
#        strnew = out[:-2]+',"VCData":' + str(j)+"}]"
#
#        text_file = open(os.path.join(folder_path, sFilename), "w")
#        n = text_file.write(strnew.replace("'",'"').replace(" ",""))
#        text_file.close()
#
#        #
#        #display(j)
#
#        #break


## Code stopped working, so testing how to create JSON file again, this time piecing it together manually

In [27]:
j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
                    .apply(lambda x: x[dReportingColumns_Data_jsonNames]
                    .to_dict())
                    .reset_index()
                    .rename(columns={0:'VCData'})
    )
                    
j

,index,S,F,Fa,NaN
0,0,WFRC_8476,0,0,NaN


In [28]:
df = dfOutputDataNew[dReportingColumns_Seg_jsonNames].drop_duplicates()
df

,S,F,Fa
9542,WFRC_8476,0,0


In [29]:
out = df.to_json(orient='records')[1:-1].replace('},{', '} {')
out

'{"S":"WFRC_8476","F":"0","Fa":"0"}'

In [30]:
j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
                    .apply(lambda x: x[dReportingColumns_Data_jsonNames])
    ).to_dict('records')                   
j

[{'VCG': 'AnnWkAM',
  'FG': 'Arterial',
  'FGF': '1.0',
  'SG': 'COU',
  'SF': '1.0',
  'DOW': '1.09',
  'PF': '0.185',
  'HF': '0.403',
  'D': 'D1',
  'DF': '0.75',
  'MD': '10.4',
  'HV': '1.3',
  'TF': '1.027',
  'PHF': 0.92,
  'LN': 1,
  'AT': 4,
  'FT': 3,
  'CP': '850',
  'PrdF': '0',
  'PkF': '0',
  '15F': '0',
  'PrdVC': '0.00',
  'PkVC': '0.00',
  '15VC': '0.00'},
 {'VCG': 'AnnWkAM',
  'FG': 'Arterial',
  'FGF': '1.0',
  'SG': 'COU',
  'SF': '1.0',
  'DOW': '1.09',
  'PF': '0.185',
  'HF': '0.403',
  'D': 'D2',
  'DF': '0.25',
  'MD': '13.8',
  'HV': '5.4',
  'TF': '1.055',
  'PHF': 0.92,
  'LN': 1,
  'AT': 4,
  'FT': 3,
  'CP': '850',
  'PrdF': '0',
  'PkF': '0',
  '15F': '0',
  'PrdVC': '0.00',
  'PkVC': '0.00',
  '15VC': '0.00'},
 {'VCG': 'AnnWkMD',
  'FG': 'Arterial',
  'FGF': '1.0',
  'SG': 'COU',
  'SF': '1.0',
  'DOW': '1.09',
  'PF': '0.294',
  'HF': '0.196',
  'D': 'D1',
  'DF': '0.641',
  'MD': '15.2',
  'HV': '5.7',
  'TF': '1.059',
  'PHF': 0.92,
  'LN': 1,
  'AT':

In [31]:
df = dfOutputDataNew[dReportingColumns_Seg_jsonNames].drop_duplicates()

out = df.to_json(orient='records')

j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
                    .apply(lambda x: x[dReportingColumns_Data_jsonNames])
    ).to_dict('records')                   
#ranslation= {39: None}

strnew = out[:-2]+',"VCData":' + str(j)+"}]"

print(strnew.replace("'",'"').replace(" ",""))

text_file = open(os.path.join(dirResults,"test.json"), "w")
n = text_file.write(strnew.replace("'",'"').replace(" ",""))
text_file.close()

[{"S":"WFRC_8476","F":"0","Fa":"0","VCData":[{"VCG":"AnnWkAM","FG":"Arterial","FGF":"1.0","SG":"COU","SF":"1.0","DOW":"1.09","PF":"0.185","HF":"0.403","D":"D1","DF":"0.75","MD":"10.4","HV":"1.3","TF":"1.027","PHF":0.92,"LN":1,"AT":4,"FT":3,"CP":"850","PrdF":"0","PkF":"0","15F":"0","PrdVC":"0.00","PkVC":"0.00","15VC":"0.00"},{"VCG":"AnnWkAM","FG":"Arterial","FGF":"1.0","SG":"COU","SF":"1.0","DOW":"1.09","PF":"0.185","HF":"0.403","D":"D2","DF":"0.25","MD":"13.8","HV":"5.4","TF":"1.055","PHF":0.92,"LN":1,"AT":4,"FT":3,"CP":"850","PrdF":"0","PkF":"0","15F":"0","PrdVC":"0.00","PkVC":"0.00","15VC":"0.00"},{"VCG":"AnnWkMD","FG":"Arterial","FGF":"1.0","SG":"COU","SF":"1.0","DOW":"1.09","PF":"0.294","HF":"0.196","D":"D1","DF":"0.641","MD":"15.2","HV":"5.7","TF":"1.059","PHF":0.92,"LN":1,"AT":4,"FT":3,"CP":"850","PrdF":"0","PkF":"0","15F":"0","PrdVC":"0.00","PkVC":"0.00","15VC":"0.00"},{"VCG":"AnnWkMD","FG":"Arterial","FGF":"1.0","SG":"COU","SF":"1.0","DOW":"1.09","PF":"0.294","HF":"0.196","D":"

# Seg WebApp Data

In [32]:
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
5,RTP50,2050 Fiscally-Constrained Scenario,2050,RTP,v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,RTP42,NaN
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
8,NB5028,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50
9,NB4232,2042 SE on a 2032 Network,2042,NoBuild32,v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed...,Yes,RTP32,RTP42


In [33]:
#column names from TDM
nameSegID   = 'SEGID'
nameFG      = 'FUNCGROUP'
nameLinks   = 'LINKS'
nameLanes   = 'LANES'
nameFT      = 'FT'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

nameDir     = 'Dir'

indexMelt   = [nameSegID,nameFG,'D1_' + nameFT,'D2_' + nameFT,'D1_' + nameLanes,'D2_' + nameLanes]

df_TDMcmb = pd.DataFrame() 

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Importing')

    csvFileName = os.path.join(dirProcessed, row.SegSummaryCSV)
    #csvFileName = os.path.splitext(dbfFileName)[0]+'.csv'

    df_TDM_import = pd.read_csv(csvFileName)

    #Remove where Total FG
    df_TDM_import = df_TDM_import[df_TDM_import.FUNCGROUP != 'Total']
    #display(df_TDM_import)
        
    print(row.loc[nameS] + ' Processing.')

    #normalize data with period/volume type fields
    df_TDM_melt = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=['D1_LINKS','D2_LINKS'])
    df_TDM_melt[nameDir] = df_TDM_melt['variable'].str.get(0) + df_TDM_melt['variable'].str.get(1)
    
    df_TDM_melt = df_TDM_melt.rename(columns={"value": nameLinks})
    
    #display(df_TDM_melt)
    
    #calculate direction level links, lanes, capacity
   
    df_TDM_melt[nameLanes] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameLanes], df_TDM_melt['D2_'+nameLanes])
    df_TDM_melt[nameFT] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameFT], df_TDM_melt['D2_'+nameFT])
    
    df_TDM_melt = df_TDM_melt.drop(columns=['variable','D1_'+nameLanes,'D2_'+nameLanes,'D1_'+nameFT,'D2_'+nameFT])
    #display(df_TDM_melt)

    df_TDM_melt['ScenarioCode'] = row.loc['ScenarioCode']
    
    df_TDM_meltn = df_TDM_melt[['ScenarioCode','SEGID','FUNCGROUP','Dir','FT','LANES']]
    
    df_TDMcmb = pd.concat([df_TDMcmb,df_TDM_meltn],ignore_index=True)
    
print('Done')

2019 Base CSV Importing
2019 Base Processing.
2023 Base CSV Importing
2023 Base Processing.
2028 TIP Projects Scenario CSV Importing
2028 TIP Projects Scenario Processing.
2032 Fiscally-Constrained Scenario CSV Importing
2032 Fiscally-Constrained Scenario Processing.
2042 Fiscally-Constrained Scenario CSV Importing
2042 Fiscally-Constrained Scenario Processing.
2050 Fiscally-Constrained Scenario CSV Importing
2050 Fiscally-Constrained Scenario Processing.
2032 SE on a 2028 Network Scenario CSV Importing
2032 SE on a 2028 Network Scenario Processing.
2042 SE on a 2028 Network Scenario CSV Importing
2042 SE on a 2028 Network Scenario Processing.
2050 SE on a 2028 Network Scenario CSV Importing
2050 SE on a 2028 Network Scenario Processing.
2042 SE on a 2032 Network CSV Importing
2042 SE on a 2032 Network Processing.
2050 SE on a 2042 Network CSV Importing
2050 SE on a 2042 Network Processing.
2050 SE on a 2042 Network CSV Importing
2050 SE on a 2042 Network Processing.
2032 Needs-Based S

In [34]:
df_TDMcmb

,ScenarioCode,SEGID,FUNCGROUP,Dir,FT,LANES
0,Base19,0006_141.0,Arterial,D1,2.0,1.0
1,Base19,0006_141.0,Freeway,D1,0.0,0.0
2,Base19,0006_141.0,Managed,D1,0.0,0.0
3,Base19,0006_141.0,CD Road,D1,0.0,0.0
4,Base19,0006_146.9,Arterial,D1,2.0,1.0
...,...,...,...,...,...,...
588179,UF50_MAG,WFRC_8473,CD Road,D2,0.0,0.0
588180,UF50_MAG,WFRC_8476,Arterial,D2,3.0,1.0
588181,UF50_MAG,WFRC_8476,Freeway,D2,0.0,0.0
588182,UF50_MAG,WFRC_8476,Managed,D2,0.0,0.0


In [35]:
df_SegIDsAll = df_TDMcmb.groupby(['SEGID'], as_index=False).agg({'ScenarioCode':[np.size]})
df_SegIDsAll.columns = ['SEGID','numRecords']
df_SegIDsAll


,SEGID,numRecords
0,0006_141.0,128
1,0006_146.9,128
2,0006_149.9,128
3,0006_150.6,128
4,0006_152.6,128
...,...,...
5065,WFRC_8470,24
5066,WFRC_8471,80
5067,WFRC_8472,24
5068,WFRC_8473,128


In [36]:
import itertools

#hours within period, for capacity
dFG = {
     'FUNCGROUP'     : ['Arterial','Freeway','Managed','CD Road']
}
df_FG = pd.DataFrame(dFG)

#hours within period, for capacity
dDir = {
     'Dir' : ['D1','D2']
}
df_Dir = pd.DataFrame(dDir)


#create df with all possible season and day-of-weak/peak groups
df_SegIDsFGAll = pd.DataFrame(list(itertools.product(df_SegIDsAll['SEGID'],df_FG['FUNCGROUP'],df_Scenarios['ScenarioCode'],df_Dir['Dir'])),columns=['SEGID','FUNCGROUP','ScenarioCode','Dir'])
display(df_SegIDsFGAll)

df_ScSegFGDir = pd.DataFrame.merge(df_SegIDsFGAll,df_TDMcmb,on=('ScenarioCode','SEGID','FUNCGROUP','Dir'),how='left')
#display(df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']==exportscenariocode])

,SEGID,FUNCGROUP,ScenarioCode,Dir
0,0006_141.0,Arterial,Base19,D1
1,0006_141.0,Arterial,Base19,D2
2,0006_141.0,Arterial,Base23,D1
3,0006_141.0,Arterial,Base23,D2
4,0006_141.0,Arterial,TIP28,D1
...,...,...,...,...
648955,WFRC_8476,CD Road,UF42,D2
648956,WFRC_8476,CD Road,UF50,D1
648957,WFRC_8476,CD Road,UF50,D2
648958,WFRC_8476,CD Road,UF50_MAG,D1


In [37]:
df_ScSegFGDir.groupby(['ScenarioCode'],as_index=False).agg({'SEGID':[np.size]})

,ScenarioCode,SEGID
,,size
0,Base19,40560
1,Base23,40560
2,NB3228,40560
3,NB4228,40560
4,NB4232,40560
5,NB5028,40560
6,NB5032,40560
7,NB5042,40560
8,RTP32,40560


In [38]:
df_ScSegFGDir_nonbase = df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']!='Base19']
display(df_ScSegFGDir_nonbase)

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES
2,0006_141.0,Arterial,Base23,D1,2.0,1.0
3,0006_141.0,Arterial,Base23,D2,2.0,1.0
4,0006_141.0,Arterial,TIP28,D1,2.0,1.0
5,0006_141.0,Arterial,TIP28,D2,2.0,1.0
6,0006_141.0,Arterial,RTP32,D1,2.0,1.0
...,...,...,...,...,...,...
648955,WFRC_8476,CD Road,UF42,D2,0.0,0.0
648956,WFRC_8476,CD Road,UF50,D1,0.0,0.0
648957,WFRC_8476,CD Road,UF50,D2,0.0,0.0
648958,WFRC_8476,CD Road,UF50_MAG,D1,0.0,0.0


In [39]:
df_ScenariosToPrevious = df_Scenarios[['ScenarioCode','PrvScenarioCode']]
df_ScSegFGDir_nonbase_wpreviouscode = pd.DataFrame.merge(df_ScSegFGDir_nonbase,df_ScenariosToPrevious,on='ScenarioCode')
display(df_ScSegFGDir_nonbase_wpreviouscode)

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,PrvScenarioCode
0,0006_141.0,Arterial,Base23,D1,2.0,1.0,Base19
1,0006_141.0,Arterial,Base23,D2,2.0,1.0,Base19
2,0006_141.0,Freeway,Base23,D1,0.0,0.0,Base19
3,0006_141.0,Freeway,Base23,D2,0.0,0.0,Base19
4,0006_141.0,Managed,Base23,D1,0.0,0.0,Base19
...,...,...,...,...,...,...,...
608395,WFRC_8476,Freeway,UF50_MAG,D2,0.0,0.0,UF42_MAG
608396,WFRC_8476,Managed,UF50_MAG,D1,0.0,0.0,UF42_MAG
608397,WFRC_8476,Managed,UF50_MAG,D2,0.0,0.0,UF42_MAG
608398,WFRC_8476,CD Road,UF50_MAG,D1,0.0,0.0,UF42_MAG


In [40]:
df_ScSegFGDir_nonbase_wpreviouscode[df_ScSegFGDir_nonbase_wpreviouscode['PrvScenarioCode']==0]

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,PrvScenarioCode


In [41]:
df_ScSegFGDir_nonbase_wpreviousdata = pd.DataFrame.merge(df_ScSegFGDir_nonbase_wpreviouscode,df_ScSegFGDir,left_on=('SEGID','FUNCGROUP','PrvScenarioCode','Dir'),right_on=('SEGID','FUNCGROUP','ScenarioCode','Dir'),how='left')
df_ScSegFGDir_nonbase_wpreviousdata = df_ScSegFGDir_nonbase_wpreviousdata[['SEGID','ScenarioCode_x','ScenarioCode_y','FUNCGROUP','Dir','FT_x','FT_y','LANES_x','LANES_y']]
df_ScSegFGDir_nonbase_wpreviousdata = df_ScSegFGDir_nonbase_wpreviousdata.fillna(0)
display(df_ScSegFGDir_nonbase_wpreviousdata)

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y
0,0006_141.0,Base23,Base19,Arterial,D1,2.0,2.0,1.0,1.0
1,0006_141.0,Base23,Base19,Arterial,D2,2.0,2.0,1.0,1.0
2,0006_141.0,Base23,Base19,Freeway,D1,0.0,0.0,0.0,0.0
3,0006_141.0,Base23,Base19,Freeway,D2,0.0,0.0,0.0,0.0
4,0006_141.0,Base23,Base19,Managed,D1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
608395,WFRC_8476,UF50_MAG,0,Freeway,D2,0.0,0.0,0.0,0.0
608396,WFRC_8476,UF50_MAG,0,Managed,D1,0.0,0.0,0.0,0.0
608397,WFRC_8476,UF50_MAG,0,Managed,D2,0.0,0.0,0.0,0.0
608398,WFRC_8476,UF50_MAG,0,CD Road,D1,0.0,0.0,0.0,0.0


In [42]:
df_ScSegFGDir_nonbase_wpreviousdata[(df_ScSegFGDir_nonbase_wpreviousdata['ScenarioCode_y']==0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y
567840,0006_141.0,UF50_MAG,0,Arterial,D1,2.0,0.0,1.0,0.0
567841,0006_141.0,UF50_MAG,0,Arterial,D2,2.0,0.0,1.0,0.0
567842,0006_141.0,UF50_MAG,0,Freeway,D1,0.0,0.0,0.0,0.0
567843,0006_141.0,UF50_MAG,0,Freeway,D2,0.0,0.0,0.0,0.0
567844,0006_141.0,UF50_MAG,0,Managed,D1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
608395,WFRC_8476,UF50_MAG,0,Freeway,D2,0.0,0.0,0.0,0.0
608396,WFRC_8476,UF50_MAG,0,Managed,D1,0.0,0.0,0.0,0.0
608397,WFRC_8476,UF50_MAG,0,Managed,D2,0.0,0.0,0.0,0.0
608398,WFRC_8476,UF50_MAG,0,CD Road,D1,0.0,0.0,0.0,0.0


In [43]:
df_ScSegFGDir_nonbase_wpreviousdata['Change_FT'] = df_ScSegFGDir_nonbase_wpreviousdata['FT_x'      ] - df_ScSegFGDir_nonbase_wpreviousdata['FT_y'      ]
df_ScSegFGDir_nonbase_wpreviousdata['Change_LN'] = df_ScSegFGDir_nonbase_wpreviousdata['LANES_x'   ] - df_ScSegFGDir_nonbase_wpreviousdata['LANES_y'   ]
#df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']] = df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']].mask(df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']] < 0, 0)
df_ScSegFGDir_nonbase_wpreviousdata

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN
0,0006_141.0,Base23,Base19,Arterial,D1,2.0,2.0,1.0,1.0,0.0,0.0
1,0006_141.0,Base23,Base19,Arterial,D2,2.0,2.0,1.0,1.0,0.0,0.0
2,0006_141.0,Base23,Base19,Freeway,D1,0.0,0.0,0.0,0.0,0.0,0.0
3,0006_141.0,Base23,Base19,Freeway,D2,0.0,0.0,0.0,0.0,0.0,0.0
4,0006_141.0,Base23,Base19,Managed,D1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
608395,WFRC_8476,UF50_MAG,0,Freeway,D2,0.0,0.0,0.0,0.0,0.0,0.0
608396,WFRC_8476,UF50_MAG,0,Managed,D1,0.0,0.0,0.0,0.0,0.0,0.0
608397,WFRC_8476,UF50_MAG,0,Managed,D2,0.0,0.0,0.0,0.0,0.0,0.0
608398,WFRC_8476,UF50_MAG,0,CD Road,D1,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
#delete lines with all zeros

#df_Changes = df_ScSegFGDir_nonbase_wpreviousdata.copy()

df_Changes = df_ScSegFGDir_nonbase_wpreviousdata[(df_ScSegFGDir_nonbase_wpreviousdata['Change_FT']!=0) | (df_ScSegFGDir_nonbase_wpreviousdata['Change_LN']!=0)].copy()

#df_ScSegFGDir_nonbase_wpreviousdata

In [45]:
df_Changes['Change_LN_Text'] = df_Changes['Change_LN'].abs().astype(str)
df_Changes['Change_LN_Text'] = df_Changes['Change_LN_Text'].replace(to_replace = "\.0+$",value = "", regex = True)
df_Changes['Change_LN_Description'] = df_Changes.apply(lambda row: row['Change_LN_Text'] + " " + row['FUNCGROUP'].lower() + " lanes added" if row['Change_LN'] > 1 else "1 " + row['FUNCGROUP'].lower() + " lane added" if row['Change_LN'] == 1 else row['Change_LN_Text'] + " " + row['FUNCGROUP'].lower() + " lanes removed" if row['Change_LN'] < -1 else "1 " + row['FUNCGROUP'].lower() + " lane removed" if row['Change_LN'] == -1 else "", axis=1)

In [46]:
df_Changes[df_Changes['Change_LN']<0]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description
4768,0089_398.7,Base23,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4769,0089_398.7,Base23,Base19,Arterial,D2,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4776,0089_399.7,Base23,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4777,0089_399.7,Base23,Base19,Arterial,D2,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4784,0089_400.5,Base23,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
559641,MAG_6172,UF50,UF42,Arterial,D2,0.0,13.0,0.0,2.0,-13.0,-2.0,2,2 arterial lanes removed
561784,MAG_6470,UF50,UF42,Arterial,D1,4.0,4.0,1.0,2.0,0.0,-1.0,1,1 arterial lane removed
561785,MAG_6470,UF50,UF42,Arterial,D2,4.0,4.0,1.0,2.0,0.0,-1.0,1,1 arterial lane removed
561792,MAG_6471,UF50,UF42,Arterial,D1,4.0,4.0,1.0,2.0,0.0,-1.0,1,1 arterial lane removed


In [47]:
df_Changes[df_Changes['SEGID']=='0089_398.7']

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description
4768,0089_398.7,Base23,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4769,0089_398.7,Base23,Base19,Arterial,D2,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4770,0089_398.7,Base23,Base19,Freeway,D1,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added
4771,0089_398.7,Base23,Base19,Freeway,D2,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added
572608,0089_398.7,UF50_MAG,0,Arterial,D1,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added
572609,0089_398.7,UF50_MAG,0,Arterial,D2,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added
572610,0089_398.7,UF50_MAG,0,Freeway,D1,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added
572611,0089_398.7,UF50_MAG,0,Freeway,D2,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added


In [48]:
df_Changes['Change_FT_Text'] = df_Changes['Change_FT'].abs().astype(str)
df_Changes['Change_FT_Description'] = df_Changes.apply(lambda row: row['FUNCGROUP'].lower() + " operational changes" if (row['Change_FT'] > 0) & (row['FT_y'] > 0) else row['FUNCGROUP'].lower() + " operational changes" if (row['Change_FT'] < 0) & (row['FT_y'] > 0) else "", axis=1)

In [49]:
df_Changes[(df_Changes['Change_FT']<0) & (df_Changes['FT_y']>0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description
608,0015_282.2,Base23,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes
609,0015_282.2,Base23,Base19,Arterial,D2,3.0,4.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes
616,0015_283.3,Base23,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes
617,0015_283.3,Base23,Base19,Arterial,D2,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes
4768,0089_398.7,Base23,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed,10.0,arterial operational changes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566457,WFRC_8294,UF50,UF42,Arterial,D2,3.0,4.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes
566464,WFRC_8295,UF50,UF42,Arterial,D1,3.0,4.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes
566465,WFRC_8295,UF50,UF42,Arterial,D2,3.0,4.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes
567552,WFRC_8436,UF50,UF42,Arterial,D1,4.0,5.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes


In [50]:
df_Changes_wDirections = pd.DataFrame.merge(df_Changes,df_Directions,on='Dir',how='left')

df_Changes_wDirections['Change_Conjunction'] = df_Changes_wDirections.apply(lambda row: ' and ' if (row['Change_FT_Description'] != '') & (row['Change_LN_Description'] != '') else '', axis=1)

df_Changes_wDirections['Change_Text'] = df_Changes_wDirections['DirDesc'] + ": " + df_Changes_wDirections['Change_LN_Description'] + df_Changes_wDirections['Change_Conjunction']  + df_Changes_wDirections['Change_FT_Description'] 
df_Changes_wDirections

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description,DirDesc,Change_Conjunction,Change_Text
0,0006_159.8,Base23,Base19,Arterial,D1,2.0,2.0,2.0,1.0,0.0,1.0,1,1 arterial lane added,0.0,,NB/EB,,NB/EB: 1 arterial lane added
1,0006_159.8,Base23,Base19,Arterial,D2,2.0,2.0,2.0,1.0,0.0,1.0,1,1 arterial lane added,0.0,,SB/WB,,SB/WB: 1 arterial lane added
2,0015_278.5,Base23,Base19,Freeway,D2,34.0,34.0,5.0,4.0,0.0,1.0,1,1 freeway lane added,0.0,,SB/WB,,SB/WB: 1 freeway lane added
3,0015_279.6,Base23,Base19,Freeway,D1,33.0,33.0,5.0,3.0,0.0,2.0,2,2 freeway lanes added,0.0,,NB/EB,,NB/EB: 2 freeway lanes added
4,0015_279.6,Base23,Base19,Freeway,D2,33.0,33.0,5.0,3.0,0.0,2.0,2,2 freeway lanes added,0.0,,SB/WB,,SB/WB: 2 freeway lanes added
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18447,WFRC_8472,UF50_MAG,0,Arterial,D2,5.0,0.0,1.0,0.0,5.0,1.0,1,1 arterial lane added,5.0,,SB/WB,,SB/WB: 1 arterial lane added
18448,WFRC_8473,UF50_MAG,0,Arterial,D1,5.0,0.0,1.0,0.0,5.0,1.0,1,1 arterial lane added,5.0,,NB/EB,,NB/EB: 1 arterial lane added
18449,WFRC_8473,UF50_MAG,0,Arterial,D2,5.0,0.0,1.0,0.0,5.0,1.0,1,1 arterial lane added,5.0,,SB/WB,,SB/WB: 1 arterial lane added
18450,WFRC_8476,UF50_MAG,0,Arterial,D1,3.0,0.0,1.0,0.0,3.0,1.0,1,1 arterial lane added,3.0,,NB/EB,,NB/EB: 1 arterial lane added


In [51]:
df_Changes_wDirections[(df_Changes_wDirections['Change_FT']<0) & (df_Changes_wDirections['Change_LN']>0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description,DirDesc,Change_Conjunction,Change_Text
5,0015_282.2,Base23,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
11,0015_283.3,Base23,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
12,0015_283.3,Base23,Base19,Arterial,D2,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
126,0194_002.4,Base23,Base19,Arterial,D2,3.0,14.0,3.0,2.0,-11.0,1.0,1,1 arterial lane added,11.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
215,2878_005.9,Base23,Base19,Arterial,D1,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7776,MAG_6840,UF50,UF42,Arterial,D2,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
7779,MAG_6842,UF50,UF42,Arterial,D1,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
7780,MAG_6842,UF50,UF42,Arterial,D2,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
7785,MAG_6848,UF50,UF42,Arterial,D1,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...


In [52]:
df_Changes_HTML = df_Changes_wDirections.groupby(['SEGID','ScenarioCode_x'])['Change_Text'].apply('<br/>'.join).reset_index()
df_Changes_HTML = pd.DataFrame.merge(df_Changes_HTML,df_Scenarios,left_on='ScenarioCode_x',right_on='ScenarioCode',how='left')
df_Changes_HTML['Change_HTML'] = "<b>" + df_Changes_HTML['ScenarioName'] + ":</b><br/>" + df_Changes_HTML['Change_Text']
df_Changes_HTML

,SEGID,ScenarioCode_x,Change_Text,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode,Change_HTML
0,0006_141.0,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...
1,0006_146.9,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...
2,0006_149.9,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...
3,0006_150.6,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...
4,0006_152.6,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...
...,...,...,...,...,...,...,...,...,...,...,...,...
8786,WFRC_8472,UF42,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF42,2042 Needs-Based Scenario,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42,<b>2042 Needs-Based Scenario:</b><br/>NB/EB: 1...
8787,WFRC_8472,UF50,NB/EB: arterial operational changes<br/>SB/WB:...,UF50,2050 Needs-Based Scenario,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50,<b>2050 Needs-Based Scenario:</b><br/>NB/EB: a...
8788,WFRC_8472,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...
8789,WFRC_8473,UF50_MAG,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50,<b>2050 Needs-Based Scenario - MAG:</b><br/>NB...


In [53]:
df_ScSegFGDir

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES
0,0006_141.0,Arterial,Base19,D1,2.0,1.0
1,0006_141.0,Arterial,Base19,D2,2.0,1.0
2,0006_141.0,Arterial,Base23,D1,2.0,1.0
3,0006_141.0,Arterial,Base23,D2,2.0,1.0
4,0006_141.0,Arterial,TIP28,D1,2.0,1.0
...,...,...,...,...,...,...
648955,WFRC_8476,CD Road,UF42,D2,0.0,0.0
648956,WFRC_8476,CD Road,UF50,D1,0.0,0.0
648957,WFRC_8476,CD Road,UF50,D2,0.0,0.0
648958,WFRC_8476,CD Road,UF50_MAG,D1,0.0,0.0


In [54]:
df_ScSegFGDir_base = df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']=='Base19']
#display(df_ScSegFGDir_base)

df_ScSegFGDir_baseNoZero = df_ScSegFGDir_base[df_ScSegFGDir_base['LANES']>0]

df_FTs = pd.read_csv(os.path.join(dirTDM, r'FTs.csv'))
df_ScSegFGDir_baseNoZero = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero,df_FTs,on='FT',how='left')

df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero['LANES'].abs().astype(str)
df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero['LANES_Text'].replace(to_replace = "\.0+$",value = "", regex = True)

df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero.apply(lambda row: row['LANES_Text'] + " lanes" if row['LANES'] > 1 else "1 lane" if row['LANES'] == 1 else "", axis=1)

df_ScSegFGDir_baseNoZero_wDirections = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero,df_Directions,on='Dir',how='left')
display(df_ScSegFGDir_baseNoZero_wDirections)

df_ScSegFGDir_baseNoZero_wDirections['FT_Text'] = df_ScSegFGDir_baseNoZero_wDirections['DirDesc'] + " " + df_ScSegFGDir_baseNoZero_wDirections['FUNCGROUP'] + " (" + df_ScSegFGDir_baseNoZero_wDirections['FT_Description'] + ")"
df_ScSegFGDir_baseNoZero_wDirections = df_ScSegFGDir_baseNoZero_wDirections.fillna("")
df_ScSegFGDir_baseNoZero_wDirections['FG_LANES'] = df_ScSegFGDir_baseNoZero_wDirections['FT_Text']  + ": " + df_ScSegFGDir_baseNoZero_wDirections['LANES_Text']
display(df_ScSegFGDir_baseNoZero_wDirections)

df_ScSegFGDir_baseNoZero_collapse = df_ScSegFGDir_baseNoZero_wDirections.groupby(['SEGID','ScenarioCode'])['FG_LANES'].apply('<br/>'.join).reset_index()

df_ScSegFGDir_baseNoZero_collapse_wScenarioName = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero_collapse, df_Scenarios, on='ScenarioCode',how='left')
df_ScSegFGDir_baseNoZero_collapse_wScenarioName['BaseDesc_HTML'] = "<b>" + df_ScSegFGDir_baseNoZero_collapse_wScenarioName['ScenarioName'] + ":</b><br/>" + df_ScSegFGDir_baseNoZero_collapse_wScenarioName['FG_LANES']

df_ScSegFGDir_baseHTML = df_ScSegFGDir_baseNoZero_collapse_wScenarioName[['SEGID','BaseDesc_HTML']]
display(df_ScSegFGDir_baseHTML)


,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,FT_Description,LANES_Text,DirDesc
0,0006_141.0,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB
1,0006_141.0,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB
2,0006_146.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB
3,0006_146.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB
4,0006_149.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB
...,...,...,...,...,...,...,...,...,...
8574,WFRC_8466,Arterial,Base19,D2,3.0,2.0,Minor Arterial,2 lanes,SB/WB
8575,WFRC_8473,Arterial,Base19,D1,5.0,1.0,Minor Collector,1 lane,NB/EB
8576,WFRC_8473,Arterial,Base19,D2,5.0,1.0,Minor Collector,1 lane,SB/WB
8577,WFRC_8476,Arterial,Base19,D1,3.0,1.0,Minor Arterial,1 lane,NB/EB


,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,FT_Description,LANES_Text,DirDesc,FT_Text,FG_LANES
0,0006_141.0,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB,NB/EB Arterial (Principal Arterial),NB/EB Arterial (Principal Arterial): 1 lane
1,0006_141.0,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB,SB/WB Arterial (Principal Arterial),SB/WB Arterial (Principal Arterial): 1 lane
2,0006_146.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB,NB/EB Arterial (Principal Arterial),NB/EB Arterial (Principal Arterial): 1 lane
3,0006_146.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB,SB/WB Arterial (Principal Arterial),SB/WB Arterial (Principal Arterial): 1 lane
4,0006_149.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB,NB/EB Arterial (Principal Arterial),NB/EB Arterial (Principal Arterial): 1 lane
...,...,...,...,...,...,...,...,...,...,...,...
8574,WFRC_8466,Arterial,Base19,D2,3.0,2.0,Minor Arterial,2 lanes,SB/WB,SB/WB Arterial (Minor Arterial),SB/WB Arterial (Minor Arterial): 2 lanes
8575,WFRC_8473,Arterial,Base19,D1,5.0,1.0,Minor Collector,1 lane,NB/EB,NB/EB Arterial (Minor Collector),NB/EB Arterial (Minor Collector): 1 lane
8576,WFRC_8473,Arterial,Base19,D2,5.0,1.0,Minor Collector,1 lane,SB/WB,SB/WB Arterial (Minor Collector),SB/WB Arterial (Minor Collector): 1 lane
8577,WFRC_8476,Arterial,Base19,D1,3.0,1.0,Minor Arterial,1 lane,NB/EB,NB/EB Arterial (Minor Arterial),NB/EB Arterial (Minor Arterial): 1 lane


,SEGID,BaseDesc_HTML
0,0006_141.0,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
1,0006_146.9,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
2,0006_149.9,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
3,0006_150.6,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
4,0006_152.6,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Ar...
...,...,...
4222,WFRC_8457,<b>2019 Base:</b><br/>NB/EB Arterial (Major Co...
4223,WFRC_8461,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Co...
4224,WFRC_8466,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Ar...
4225,WFRC_8473,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Co...


In [55]:
df_ScSegFGDir_baseNoZero

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,FT_Description,LANES_Text
0,0006_141.0,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane
1,0006_141.0,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane
2,0006_146.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane
3,0006_146.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane
4,0006_149.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane
...,...,...,...,...,...,...,...,...
8574,WFRC_8466,Arterial,Base19,D2,3.0,2.0,Minor Arterial,2 lanes
8575,WFRC_8473,Arterial,Base19,D1,5.0,1.0,Minor Collector,1 lane
8576,WFRC_8473,Arterial,Base19,D2,5.0,1.0,Minor Collector,1 lane
8577,WFRC_8476,Arterial,Base19,D1,3.0,1.0,Minor Arterial,1 lane


In [56]:
df_ScSegFGDir_base

,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES
0,0006_141.0,Arterial,Base19,D1,2.0,1.0
1,0006_141.0,Arterial,Base19,D2,2.0,1.0
32,0006_141.0,Freeway,Base19,D1,0.0,0.0
33,0006_141.0,Freeway,Base19,D2,0.0,0.0
64,0006_141.0,Managed,Base19,D1,0.0,0.0
...,...,...,...,...,...,...
648865,WFRC_8476,Freeway,Base19,D2,0.0,0.0
648896,WFRC_8476,Managed,Base19,D1,0.0,0.0
648897,WFRC_8476,Managed,Base19,D2,0.0,0.0
648928,WFRC_8476,CD Road,Base19,D1,0.0,0.0


In [57]:
df_SegmentDetails = pd.DataFrame()
display(df_SegmentDetails)

for (idx, row) in df_SegIDsAll.iterrows():
    
    _segid = row.loc['SEGID']
    print("SEGID: " + _segid)

    #strFinal = "<b>Segment " + _segid + "</b><br/><br/>"
    strFinal = "<br/>"
    
    for (idx, row) in df_Scenarios.iterrows():
        _scenariocode = row.loc['ScenarioCode']
        _scenarioname = row.loc['ScenarioName']
        #print(_scenariocode)
        #print(_scenarioname)
        
        sText = "<b>" + _scenarioname + "</b><br/>"
        
        #list base conditions for base scenario
        if _scenariocode == 'Base19':
            
            #print(_scenarioname)
            
            df_base = df_ScSegFGDir_baseHTML[df_ScSegFGDir_baseHTML['SEGID']==_segid]
            
            if len(df_base.index) > 0:
                strFinal = strFinal + df_base.iloc[0]['BaseDesc_HTML'] + "<br/><br/>"
                #display(df_base.iloc[0]['BaseDesc_HTML'])
            else:
                strFinal = strFinal + "<b>No lanes in " +_scenarioname + "</b><br/><br/>"
            
            #for (idx, row) in df_base.iterrows():
                
                #print(row.loc['BaseDesc_HTML'])
        else:
            
            df_change = df_Changes_HTML[(df_Changes_HTML['SEGID']==_segid) & (df_Changes_HTML['ScenarioCode_x']==_scenariocode)]
            
            if len(df_change.index) > 0:
                #print(_scenarioname)
                strFinal = strFinal + df_change.iloc[0]['Change_HTML'] + "<br/><br/>" 
                #display(df_change.iloc[0]['Change_HTML'])
    
    #display(strFinal)
    df_SegmentDetails = df_SegmentDetails.append({'SEGID'         : _segid,
                                                  'SegDetail_HTML': strFinal
                                                 },ignore_index=True)
    df_SegmentDetails.index.name = 'ID'
    

""


SEGID: 0006_141.0
SEGID: 0006_146.9
SEGID: 0006_149.9
SEGID: 0006_150.6
SEGID: 0006_152.6
SEGID: 0006_152.9
SEGID: 0006_155.8
SEGID: 0006_155.9
SEGID: 0006_157.3
SEGID: 0006_157.6
SEGID: 0006_158.5
SEGID: 0006_158.9
SEGID: 0006_159.3
SEGID: 0006_159.6
SEGID: 0006_159.8
SEGID: 0006_160.3
SEGID: 0006_173.4
SEGID: 0006_173.7
SEGID: 0006_174.0
SEGID: 0006_174.4
SEGID: 0006_174.9
SEGID: 0006_175.6
SEGID: 0006_176.1
SEGID: 0006_177.0
SEGID: 0006_177.2
SEGID: 0006_177.5
SEGID: 0006_177.9
SEGID: 0006_178.8
SEGID: 0006_181.6
SEGID: 0006_184.1
SEGID: 0006_187.5
SEGID: 0006_194.8
SEGID: 0006_203.6
SEGID: 0006_211.2
SEGID: 0006_216.2
SEGID: 0013_000.0
SEGID: 0013_000.6
SEGID: 0013_001.3
SEGID: 0013_001.6
SEGID: 0013_001.8
SEGID: 0013_002.5
SEGID: 0013_002.9
SEGID: 0013_003.3
SEGID: 0013_004.9
SEGID: 0013_005.6
SEGID: 0015_241.6
SEGID: 0015_242.4
SEGID: 0015_244.8
SEGID: 0015_246.9
SEGID: 0015_248.8
SEGID: 0015_250.9
SEGID: 0015_251.6
SEGID: 0015_252.5
SEGID: 0015_253.6
SEGID: 0015_255.9
SEGID: 001

In [58]:
dirResults

'e:\\GitHub\\V-over-C-Calculations\\results'

In [59]:
df_SegmentDetails.to_json(os.path.join(dirResults, r'WebAppData\segmentdetails.json'),orient='records')

In [60]:
len(df_SegmentDetails.index)

5070

# Forecasts Prep

In [61]:
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
5,RTP50,2050 Fiscally-Constrained Scenario,2050,RTP,v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,RTP42,NaN
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
8,NB5028,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50
9,NB4232,2042 SE on a 2032 Network,2042,NoBuild32,v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed...,Yes,RTP32,RTP42


In [62]:
df_Forecasts = pd.read_csv(os.path.join(dirResults, r'0_Forecasts.csv'))

df_Forecasts_wScenarios = pd.DataFrame.merge(df_Forecasts,df_Scenarios,on='ScenarioCode')

df_Forecasts_wScenarios['ForecastAADT_Rounded'] = (df_Forecasts_wScenarios['ForecastAADT'] / 100).astype(int) *100

df_Forecasts_wScenarios['Forecast_Formatted'] = df_Forecasts_wScenarios['ForecastAADT_Rounded'].map('{:,.0f}'.format)

df_Forecasts_wScenarios['Forecast_HTML'] = "<b>" + df_Forecasts_wScenarios['ScenarioName'] + "</b>: " + df_Forecasts_wScenarios['Forecast_Formatted']

df_Forecasts_wScenarios_grouped = df_Forecasts_wScenarios.groupby(['SEGID'])['Forecast_HTML'].apply('<br/>'.join).reset_index()

display(df_Forecasts_wScenarios_grouped)

df_Forecasts_wScenarios_grouped.to_json(os.path.join(dirResults, r'WebAppData\forecastdetails.json'),orient='records')


,SEGID,Forecast_HTML
0,0006_141.0,"<b>2019 Base</b>: 1,500<br/><b>2023 Base</b>: ..."
1,0006_146.9,"<b>2019 Base</b>: 1,500<br/><b>2023 Base</b>: ..."
2,0006_149.9,"<b>2019 Base</b>: 2,400<br/><b>2023 Base</b>: ..."
3,0006_150.6,"<b>2019 Base</b>: 2,400<br/><b>2023 Base</b>: ..."
4,0006_152.6,"<b>2019 Base</b>: 2,400<br/><b>2023 Base</b>: ..."
...,...,...
5344,WFRC_8472,<b>2019 Base</b>: 0<br/><b>2023 Base</b>: 0<br...
5345,WFRC_8473,"<b>2019 Base</b>: 0<br/><b>2023 Base</b>: 6,90..."
5346,WFRC_8474,<b>2019 Base</b>: 0<br/><b>2023 Base</b>: 0<br...
5347,WFRC_8475,<b>2019 Base</b>: 0<br/><b>2023 Base</b>: 0<br...


In [63]:
df_Forecasts_wScenarios[df_Forecasts_wScenarios['SEGID']=='0091_002.0']

,ScenarioCode,SEGID,ForecastAADT,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode,ForecastAADT_Rounded,Forecast_Formatted,Forecast_HTML
662,Base19,0091_002.0,24041.0,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN,24000,"24,000","<b>2019 Base</b>: 24,000"
6011,Base23,0091_002.0,25500.0,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN,25500,"25,500","<b>2023 Base</b>: 25,500"
11360,TIP28,0091_002.0,26500.0,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN,26500,"26,500","<b>2028 TIP Projects Scenario</b>: 26,500"
16709,RTP32,0091_002.0,27500.0,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN,27500,"27,500","<b>2032 Fiscally-Constrained Scenario</b>: 27,500"
22058,RTP42,0091_002.0,30500.0,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN,30500,"30,500","<b>2042 Fiscally-Constrained Scenario</b>: 30,500"
27407,RTP50,0091_002.0,32500.0,2050 Fiscally-Constrained Scenario,2050,RTP,v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,RTP42,NaN,32500,"32,500","<b>2050 Fiscally-Constrained Scenario</b>: 32,500"
34461,NB3228,0091_002.0,27500.0,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32,27500,"27,500","<b>2032 SE on a 2028 Network Scenario</b>: 27,500"
38715,NB4228,0091_002.0,30500.0,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42,30500,"30,500","<b>2042 SE on a 2028 Network Scenario</b>: 30,500"
42992,NB5028,0091_002.0,32000.0,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50,32000,"32,000","<b>2050 SE on a 2028 Network Scenario</b>: 32,000"
47610,NB4232,0091_002.0,30500.0,2042 SE on a 2032 Network,2042,NoBuild32,v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed...,Yes,RTP32,RTP42,30500,"30,500","<b>2042 SE on a 2032 Network</b>: 30,500"


# Seg File Export

In [64]:
#for (idx, row) in sdf_Forecasts.iterrows():
#
#    segid = row.loc['SEGID']
#    
#    #print(segid)   
#    
#    df_export = df_VCcmb[(df_VCcmb['SEGID']==segid) & (df_VCcmb['VCGroupCode'].str.contains('DY')==False) & (df_VCcmb['VCGroupCode'].str.contains('FrPM')==False)]
#    df_export = df_export.drop(columns=['SEGID'])
#    
#    df_export_s_fg = df_export.groupby(['ScenarioCode','FUNCGROUP','AREATYPE','FT','LANES','Cap1Hr','DOWFACFC','SsnGrp','ForecastAADT'],as_index=False).agg({'VCGroupCode':[np.size]})
#    df_export_s_fg.columns = ['ScenarioCode','FUNCGROUP','AREATYPE','FT','LANES','Cap1Hr','DOWFACFC','SsnGrp','ForecastAADT','numRec']
#    df_export_s_fg_vc = df_export[['ScenarioCode','FUNCGROUP','VCGroupCode','Dir','SeasonFactor','DOWFactor','PercentTrucksMD','PercentTrucksHV','VolPercentSegDY','PHF','VolMaxHourPercent','ForecastPeakHour','ForecastPeak15MinPCE','VCPeak15MinPCE']]
#    
#    filename_s_fg = os.path.join(dirResults,r"WebAppData\segments\\" + segid + '_s_fg.csv')
#    filename_s_fg_vc = os.path.join(dirResults,r"WebAppData\segments\\" + segid + '_s_fg_vc.csv')
#    display('Exporting ' + segid)
#    df_export_s_fg.to_csv(filename_s_fg,index=False)
#    df_export_s_fg_vc.to_csv(filename_s_fg_vc,index=False)